In [11]:
from pandas.core.frame import DataFrame
import requests as req
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mongo_db import TestArchive
import sys
from pymongo import MongoClient 
from scipy.stats import ttest_ind
from scipy.stats import t

In [12]:
df = pd.read_json('../data/processed/casted_clearedDF.jsonl' , lines=True)

if len(sys.argv) != 2:
    test_size = 0.2
else : 
    test_size = float(sys.argv[1])


In [13]:
    #prepare data
    df = df.drop(columns=[
        "delivery_company",
        "session_id",
        "purchase_timestamp", 
        "delivery_timestamp",
        "product_name",
        "price",
        "category_path",
        "week_day"
        ])

    label_city = LabelEncoder()
    df['city'] = label_city.fit_transform(df['city'])
    label_street = LabelEncoder()
    df['street'] = label_street.fit_transform(df['street'])

    x = df[['street', 'purchase_week_day_plus_hour', 'city', "user_id"]]
    y = df['delivery_time']

In [14]:
#split data
_, x_test, _, y_test = train_test_split(x, y, test_size = test_size, random_state = 0)

number_of_requests = len(x_test)

x_test : DataFrame = x_test

In [15]:
#reset database before running A/B test
test_archive = TestArchive()
test_archive.reset_collections()

In [16]:

#group y values for each test group
test_group_a = []
test_group_b = []

for index, row in x_test.iterrows():
    if row['user_id'] % 2 == 0:
        test_group_a.append(y_test[index])
    else :
        test_group_b.append(y_test[index])


x_test = x_test.values.tolist()
    

In [17]:
#perform ab tests
url = 'http://localhost:8000/ab_test?user_id='

for i in range(0, number_of_requests):
    record = x_test[i]
    data = {
        'street' : record[0],
        'purchase_week_day_plus_hour' : record[1],
        'city' : record[2]
    }
    req.post(url + str(record[3]), json=data)


In [18]:
#load the results
test_archive = TestArchive()

predictions_a = []
for ele_a in test_archive.find_group('group_a'):
    predictions_a.append(ele_a['prediction'])
predictions_b = []
for ele_b in test_archive.find_group('group_b'):
    predictions_b.append(ele_b['prediction'])

In [19]:
#calculate quality metrics
predictions_a = pd.Series(predictions_a)
predictions_b = pd.Series(predictions_b)

test_group_a = pd.Series(test_group_a)
test_group_b = pd.Series(test_group_b)


print(len(predictions_a))
print(len(test_group_a))

print(len(predictions_b))
print(len(test_group_b))


640
640
767
767


In [20]:
correcntess_a = []

for index, ele in enumerate(predictions_a):
    result = 1 if ele == test_group_a[index] else 0
    correcntess_a.append(result)

    correcntess_b = []

for index, ele in enumerate(predictions_b):
    result = 1 if ele == test_group_b[index] else 0
    correcntess_b.append(result)

In [21]:
t_stat, p = ttest_ind(correcntess_b, correcntess_a)

dof = len(correcntess_b) + len(correcntess_a) - 2

print("stat_t value : " + str(t_stat))
print("p-value : " + str(t.sf(t_stat, dof)))

stat_t value : 4.255408962779513
p-value : 1.112440382678287e-05
